# typing - 형 힌트 지원

---

## [ Spectial types ]

    타입 주석으로 직접 사용되는 타입들

### Any

모든 형과 호환

In [71]:
from typing import Any

anything: Any

### NoReturn

return 값이 없을 때 활용

In [72]:
from typing import NoReturn

def stop() -> NoReturn:
    raise RuntimeError("no way")

### TypeAlias

변수에 타입을 할당함으로써 정의하며 복잡한 타입의 구조를 단순화하는데에 유용

None은 type(None)으로 자동으로 치환됨

In [73]:
from typing import TypeAlias
from collections.abc import Sequence

Factors: TypeAlias = list[int]

ConnectionOptions = dict[str, str]
Address = tuple[str, int]
Server = tuple[Address, ConnectionOptions]
ServerSequence = Sequence[Server]  # == Sequence[tuple[tuple[str, int], dict[str, str]]]

## [ Special forms ]

    타입 주석으로 사용되며 []를 활용해 추가적인 정보를 지정할 수 있는 타입들

### Tuple

In [74]:
some_tuple: tuple[int, str]

### Union

In [75]:
# 중복 무시, 순서 무시
int_str_float: int | str | float

### Optional

In [76]:
optional_int: int | None

### Callable

In [77]:
from collections.abc import Callable

any_function: Callable[..., Any]  # 호출 서명 생략
add_function: Callable[[int, int], int]

### Concatenate

Callable의 인수 부분에 사용됨

Concatenate[Arg1Type, Arg2Type, ..., ParamSpecVariable]

In [78]:
from threading import Lock
from typing import Concatenate, ParamSpec, TypeVar

P = ParamSpec('P')
R = TypeVar('R')

my_lock = Lock()

def with_lock(f: Callable[Concatenate[Lock, P], R]) -> Callable[P, R]:
    def inner(*args: P.args, **kwargs: P.kwargs) -> R:
        return f(my_lock, *args, **kwargs)
    return inner

@with_lock
def sum_threadsafe(lock: Lock, numbers: list[float]) -> float:
    with lock:
        return sum(numbers)

sum_threadsafe([1.1, 2.2, 3.3])

6.6

### Type

타입 주석으로 인스턴스가 아닌 클래스 자체를 제공하기 위해 사용

type[C]는 공변적(C 및 C의 서브 클래스 또한 허용)

In [79]:
int_instance: int
int_type: type[int] = int

### Literal

'이것들 중 하나'를 표현할 때 사용

In [80]:
from typing import Literal

boolean: Literal[True, False]
file_mode: Literal["r", "rg", "w", "wb"]

### ClassVar

클래스 변수임을 표시하기 위해 활용

In [81]:
from typing import ClassVar

class Starship:
    stats: ClassVar[dict[str, int]] = {} 
    damage: int = 10                     

enterprise_d = Starship()
enterprise_d.stats = {}  # Error!
Starship.stats = {}


### Final

정의 이후 대입 금지임을 표시하기 위해 활용

In [82]:
from typing import Final

MAX_SIZE: Final = "origin"
MAX_SIZE = "other"  # Error!

class Base:
    F: Final[int] = "origin"  # Error!

class Sub(Base):
    F = "other"  # Error!

### Annotated

타입 분석 도구(mypy같은)를 직접 다루는 게 아니라면 사용할 일이 없음

### TypeGuard

boolean을 반환하는 사용자 정의 함수의 반환 유형에 사용되는 특수 폼.

조건문에서 해당 함수의 boolean 값을 판단하여 변수의 타입 범위를 좁히는데에 사용됨

In [83]:
from typing import TypeGuard

def is_str_list(values: list[Any]) -> TypeGuard[list[str]]:
    return all(isinstance(x, str) for x in values)

def func(values: list[Any]):
    if is_str_list(values):
        values  # list[str]
        return
    values  # list

## [ Building generic types ]

    타입 주석으로 사용되지 않으며 제네릭 클래스 또는 함수 정의를 돕는 타입들

### Generic

제네릭 형을 정의하는데에 사용되는 ABC

인스턴스화된 Generic(하나 이상의 TypeVar 인스턴스를 []에 제공받은)을 상속하여 제네릭형이 선언됨

사용 시 []에 실제로 사용될 타입을 삽입하여 타입 주석을 제공받음

In [84]:
from typing import Generic, TypeVar


T = TypeVar("T")

class Node(Generic[T]): ...  # type: ignore

node = Node()  # Node[Unknown] Error!
node2: Node[int] = Node()  # 강제로 지정 가능

In [85]:
class Node(Generic[T]):
    x: T|None = None
    def __init__(self, label: T = None) -> None: ...

# TypeVar가 __new__ 또는 __init__에 사용되면 인자의 타입으로 유형이 추론됨
none_node = Node()
int_node = Node(0)
int_node.x  # int | None
str_node = Node("")
str_node.x  # str | None

# 깡으로 쓰기 보다
# TypeAlias를 쓰는 편이 가독성에 좋음
node: Node[int] = Node(1)
IntNode = Node[int]
node: IntNode = Node(1)

In [86]:
from collections.abc import Iterable, Iterator

# 공변(covariant), 반공변(contravariant), 무공변(invariant)
# https://blog.magrathealabs.com/pythons-covariance-and-contravariance-b422c63f57ac#:~:text=In%20Python%2C%20declaring%20if%20a,are%20covariant%20and%20_contra%20%2C%20contravariant.
# https://medium.com/@lazysoul/%EA%B3%B5%EB%B3%80%EA%B3%BC-%EB%B6%88%EB%B3%80-297cadba191

class Employee: ...
class Manager(Employee): ...

# 무공변 = 자신타입만 허용
T_v = TypeVar("T_v")
# 공변 = 자신타입 및 서브타입 허용, *_co
T_co = TypeVar("T_co", covariant=True)  
# 반공변 = 자신타입 및 슈퍼타입 허용, *_contra
T_contra = TypeVar("T_contra", contravariant=True)

class ImmutableList(Generic[T_co]):
    def __init__(self, items: Iterable[T_co]) -> None: ...
    def __iter__(self) -> Iterator[T_co]: ...

def dump_employees(emps: ImmutableList[Employee]) -> None: 
    for emp in emps:
        emp 

managers = ImmutableList([Manager(), Manager()])
dump_employees(managers)

# 공변 및 반공변 설정은 제네릭 클래스에서 사용되는 개념이므로 일반 함수에서는 사용 안됨
def test(x: T_co): ...

TypeError: iter() returned non-iterator of type 'NoneType'

### TypeVar

제네릭형 또는 제네릭함수에 사용되는 타입 변수

In [ ]:
from typing import TypeVar, Generic
from collections.abc import Sequence, Iterable, Sized

# 항상 대문자 변수에 직접 할당
A = TypeVar("A")

# constrained 지정으로 해당 유형으로 제약 가능
# 연산 결과는 해당 유형중 하나로 평가됨
AC = TypeVar("AC", str, bytes)

# bound 지정으로 해당 유형의 서브타입으로 제약 가능
AB = TypeVar("AB", bound=Sized)

# 제네릭 클래스에서만 사용되는 개념으로 공변 설정 가능
D = TypeVar("D")  # 무공변(자신 타입, 기본)
D_co = TypeVar("D_co", covariant=True)  # 공변(자신 및 서브 타입)
D_contra = TypeVar("D_contra", contravariant=True)  # 반공변 (자신 및 부모 타입)

In [ ]:
B = TypeVar("B")
C = TypeVar("C")

# Generic의 []에 전달되지 않은 TypeVar는 클래스 스코프 내에서 사용 불가
class MyOtherClass(Generic[B]):
    x: list[B] = []
    y: list[C] = []  # Error!

# 메서드에서는 사용 가능
class MyClass(Generic[B]):
    def meth_1(self, x: B) -> B: ...
    def meth_2(self, x: B) -> B: ...
    def not_in_generic_type_var(self, x: B, y: C) -> C: ...  # Allow

a: MyClass[int] = MyClass()  # a = MyClass[int]()
result = a.meth_1(1)
result = a.meth_2("a")  # Error!
result = a.not_in_generic_type_var(1, "string")  # str

# 제네릭에 전달된 TypeVar는 해당 클래스 스코프 내에서만 활용 가능
# 중첩된 클래스에서도 마찬가지
class Outer(Generic[C]):
    class Inner(Iterable[C]):
        ...

### ParamSpec

Callable의 매개변수 사양을 표현하는데에 사용되는 TypeVar의 특수 형태

한 Callable의 매개변수 유형을 다른 Callable에 전달하는데에 사용됨

In [ ]:
import logging


T = TypeVar('T')  # type: ignore
P = ParamSpec('P')  # type: ignore

# def add_logging(f: Callable[..., Any]) -> Callable[..., Any]:
def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    def inner(*args: P.args, **kwargs: P.kwargs):
        # P.args: ParamSpecArgs
        # P.kwargs: ParamSpecKwargs
        logging.info(f'{f.__name__} was called')
        return f(*args, **kwargs)
    return inner

@add_logging
def add_two(x: float, y: float) -> float:
    return x + y

add_two

## [ Other special directives ]

    타입 주석으로 사용되지 않으며 타입을 선언하는데에 사용되는 타입들

### NamedTuple

collections.namedtuple을 쉽게 사용

In [ ]:
from typing import NamedTuple

class Employee(NamedTuple):
    name: str
    id: int = 3
# == collections.namedtuple("Employee", ['name', 'id'])

e = Employee("홍길동")
e.name
e.id

### NewType

아무것도 없는 서브클래스를 만들어 타이핑에 사용하고 싶을 때 대신 활용

In [ ]:
from typing import NewType

# class UserID(int): ...  # 이렇게 쓸거라면
UserID = NewType("UserID", int)  # 차라리 이렇게 쓰세요

def get_user_by_id(id: UserID): ...

get_user_by_id(1)  # Error
get_user_by_id(UserID(1))

### TypedDict

타입 힌팅이 제공되는 딕셔너리

In [ ]:
from typing import TypedDict

class Point2D(TypedDict):
    x: int
    y: int
    label: str

p_a = Point2D(x=1, y=1, label="fdsa")  # == dict(x=1, y=1, label="fdsa")

# [] 여기에서도 타입 힌팅이 제공된다!
result = p_a["x"]  # int
result = p_a["y"]  # int
result = p_a["label"]  # str

## [ Generic concrete collections ]

    내장 유형

In [ ]:
d: dict[str, int]
l: list[str]
s: set[int]
fs: frozenset[str]

from collections import defaultdict, OrderedDict, ChainMap, Counter, deque

dd: defaultdict[str, int]
od: OrderedDict[str, int]
cm: ChainMap[str, int]
ct: Counter[str]
dq: deque[int]

from typing import IO

tio: IO[str]
bio: IO[bytes]

from re import Match, Pattern

tp: Pattern[str]
bp: Pattern[bytes]
tm: Match[str]
bm: Match[bytes]

## [ Abstract Base Classes ]

    collections.abc, contextlib

In [ ]:
from collections import abc
import contextlib


# Container
def container_example(x: abc.Container[int]):
    1 in x   # __contains__


# Hashable
def hashable_example(x: abc.Hashable):
    hash(x)  # __hash__


# Iterable
def iterable_example(x: abc.Iterable[int]):
    iter(x)  # __iter__


# Iterator(Iterable)
def iterator_example(x: abc.Iterator[int]):
    next(x)  # __next__


# Reversible(Iterable)
def reversible_example(x: abc.Reversible[int]):
    reversed(x)  # __reversed__


# Generator(Iterator)
# Generator[int, None, None] == Iterator[int]
def generator_example(x: abc.Generator[int, float, str]):
    x.send(0.1)         # send
    x.throw(Exception)  # throw
    x.close()           # close


# Sized
def sized_example(x: abc.Sized):
    len(x)  # __len__


# Callable
def callable_example(x: abc.Callable[[int, int], int]):
    x(1, 2)  # __call__


# Collection(Sized, Iterable, Container)
def collection_example(x: abc.Collection[int]):
    ...


# Sequence(Reversible, Collection)
def sequence_example(x: abc.Sequence[int]):
    x[1:3]      # __getitem__
    x.index(2)  # index
    x.count(1)  # count


# MutableSequence(Sequence)
def mutable_sequence_example(x: abc.MutableSequence[str]):
    x.append('item')        # append
    x.reverse()             # reverse
    x.extend(["v1", 'v2'])  # extend
    x.pop()                 # pop
    x.remove("v2")          # remove
    x += ["v4, v5"]         # __iadd__


# ByteString(Sequence)
def byte_string_example(x: abc.ByteString):
    ...


# Set(Collection)
def set_example(x: abc.Set[int], y: abc.Set[int]):
    x == y           # __eq__
    x != y           # __ne__
    x > y            # __gt__
    x >= y           # __ge__
    x < y            # __lt__
    x <= y           # __le__
    x & y            # __and__
    x | y            # __or__
    x - y            # __sub__
    x ^ y            # __xor__
    x.isdisjoint(y)  # isdisjoint


# MutableSet(Set)
def mutable_set_example(x: abc.MutableSet[int], y: abc.MutableSet[int]):
    x.clear()    # clear
    x.pop()      # pop
    x.remove(1)  # remove
    x |= y       # __ior__
    x &= y       # __iand__
    x ^= y       # __ixor__
    x -= y       # __isub__


# Mapping(Collection)
def mapping_example(x: abc.Mapping[str, int], y: abc.Mapping[str, int]):
    x.keys()
    x.values()
    x.items()
    x.get("key")
    x == y
    x != y


# MutableMapping(Mapping)
def mutable_mapping_example(x: abc.MutableMapping[str, int], y: abc.Mapping[str, int]):
    x["key"] = 1            # __setitem__
    del x['key']            # __delitem__
    x.pop("key", None)      # pop
    x.popitem()             # popitem
    x.clear()               # clear
    x.update(y)             # update
    x.setdefault("key", 0)  # setdefault


# MappingView(Sized)
def mapping_view_example(x: abc.MappingView):
    ...


# ItemsView(MappingView, Set)
def items_view_example(x: abc.ItemsView[str, str]):
    ...


# KeysView(MappingView, Set)
def keys_view_example(x: abc.KeysView[str]):
    ...


# ValuesView(MappingView, Collection)
def values_view_example(x: abc.ValuesView[str]):
    ...


# Awaitable
async def awaitable_example(x: abc.Awaitable[int]):
    await x  # __await__


# Coroutine(Awaitable)
async def coroutine_example(x: abc.Coroutine[int, str, float]):
    x.send("string")    # send
    x.throw(Exception)  # throw
    x.close()           # close


# AsyncIterable
async def async_iterable_example(x: abc.AsyncIterable[int]):
    aiter(x)  # __aiter__
    async for i in x:
        i

# AsyncIterator(AsyncIterable)
async def async_iterator_example(x: abc.AsyncIterator[int]):
    await anext(x)  # __anext__


# AsyncGenerator(AsyncIterator)
# AsyncGenerator[int, None] == AsyncIterator[int]
async def async_generator_example(x: abc.AsyncGenerator[str, int]):
    await x.asend(1)           # asend
    await x.athrow(Exception)  # athrow
    await x.aclose()           # aclose


# ContextManager
def context_manager_example(x: contextlib.AbstractContextManager[str]):
    with x as i:  # __enter__, __exit__
        i


# AsyncContextManager
async def async_context_manager_example(x: contextlib.AbstractAsyncContextManager[str]):
    async with x as i:  # __aenter__, __aexit__
        i

## [ Protocol ]

    runtime_checkable()로 사용 가능한 프로토콜들

In [ ]:
import typing

class Example:
    ...

issubclass(Example, typing.SupportsAbs)
issubclass(Example, typing.SupportsBytes)
issubclass(Example, typing.SupportsComplex)
issubclass(Example, typing.SupportsFloat)
issubclass(Example, typing.SupportsIndex)
issubclass(Example, typing.SupportsInt)
issubclass(Example, typing.SupportsRound)

## [ Function and Decorators ]

    여러 유틸성 함수 및 데코레이터

### typing.cast

정적 타입 검사기가 값을 다른 타입으로 인식하도록 변경 (쓸 일이 있을까?)

In [ ]:
import typing

x = 1.0                  # x: float
x = typing.cast(str, x)  # x: str

### typing.overload

데코레이터로 활용되며, 인자의 타입에 따라 반환 타입을 다양하게 서술하고 싶을 때 활용

In [ ]:
from typing import overload


@overload
def process(response: None) -> None: ...

@overload
def process(response: int) -> tuple[int, str]: ...

@overload
def process(response: bytes) -> str: ...

@overload
def process(response: Any) -> Any: ...

def process(response: Any) -> Any: ...

r = process(None)  # None
r = process(1)     # tuple[int, str]
r = process(b"")   # str
r = process([])    # Any

### typing.final

재정의 불가 지정(메서드는 오버라이딩 불가, 클래스는 상속 불가)

In [ ]:
from typing import final


class Base:

    @final
    def done(self) -> None: ...

class Sub(Base):

    def done(self): ...


@final
class Leaf: ...

class Other(Leaf): ...

### [기타](https://docs.python.org/ko/3/library/typing.html#functions-and-decorators)

용도를 떠올리기 어렵고 쓸 일이 거의 없다 싶은 것들은 제외